In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/dataset CHECKED/train.csv')
train.head()

,label,text
0,real,【#武汉卫健委回应集中核酸检测#】#武汉卫健委集中核酸检测十问十答#：①为什么要在全市范围内...
1,real,【#青海连续14天无新增病例#】记者从青海省卫健委获悉，​2月19日0-24时，青海省报告新...
2,real,【人民直播：#湖北疫情防控工作发布会#】目前，江汉方舱医院医疗救治工作进展如何？3月1日下午...
3,real,【#北京全市核酸检测76499人阳性59人#】在北京今天上午召开的新冠肺炎疫情防控工作新闻发...
4,fake,美国的10艘医疗军舰已经开进纽约港，每艘船上有1000张病床，上面拥有所有的医疗抢救设施，每...


In [ ]:
train

,label,text
0,real,【#武汉卫健委回应集中核酸检测#】#武汉卫健委集中核酸检测十问十答#：①为什么要在全市范围内...
1,real,【#青海连续14天无新增病例#】记者从青海省卫健委获悉，​2月19日0-24时，青海省报告新...
2,real,【人民直播：#湖北疫情防控工作发布会#】目前，江汉方舱医院医疗救治工作进展如何？3月1日下午...
3,real,【#北京全市核酸检测76499人阳性59人#】在北京今天上午召开的新冠肺炎疫情防控工作新闻发...
4,fake,美国的10艘医疗军舰已经开进纽约港，每艘船上有1000张病床，上面拥有所有的医疗抢救设施，每...
...,...,...
1678,real,【转发传递！16分钟纪录片#起底污名化中国抗疫套路#】中国努力降低新冠疫情死亡率、及时发布和...
1679,real,【#浙江新增1例境外输入性确诊病例#】据浙江省卫健委通报，2020年3月1日0-24时，浙江...
1680,real,【#黑龙江首发援鄂3000吨大米专列#】2月14日15时，编挂49辆满载3000吨三江平原大...
1681,real,【齐心“鲁”力！#山东1775位援鄂医务人员全名单#】十二批逆行者，#每个名字都闪闪发亮#！...


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/NLP Project/Datasets/dataset CHECKED/test.csv')
test.head()

,label,text
0,real,【基因研究：#巴西流行的新冠病毒与欧美样本相似#】巴西奥斯瓦尔多·克鲁斯基金会日前公布一份研...
1,real,【#北京快递小哥外卖骑手全员核酸检测#】据北京日报，北京快递企业、外卖企业近日陆续安排快递员...
2,real,【继续加油！全国#连续5天治愈出院超千人#】从12日至16日，全国连续治愈出院人数分别为11...
3,real,【#民警抗疫一线奋战16天牺牲#】12日，江苏徐州公安局交警支队指导员司元羽连续奋战防疫检查...
4,real,【#全球新冠肺炎超1200万例#】根据美国约翰斯·霍普金斯大学最新统计数据，全球新冠肺炎确诊...


In [ ]:
#encode output
label_mapping = {'fake': 1, 'real': 0}
train.label = train.label.map(label_mapping)
test.label = test.label.map(label_mapping)

### Train val split

In [ ]:
train = train.dropna()
test = test.dropna()
X_train, X_val, y_train, y_val = train_test_split(train['text'], train['label'], test_size=0.2, random_state=42)
X_test, y_test = test['text'], test.label

#### Preprocessing the dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [ ]:
X_train_encoded = tokenizer(
    list(X_train.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_val_encoded = tokenizer(
    list(X_val.values),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
X_test_encoded = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = FakeNewsDataset(X_train_encoded, y_train.values)
val_dataset = FakeNewsDataset(X_val_encoded, y_val.values)
test_dataset = FakeNewsDataset(X_test_encoded, y_test.values)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

## Training

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00


In [ ]:
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification
import evaluate
import numpy as np
import os
os.environ['HF_MLFLOW_LOG_ARTIFACTS'] = "1" # save models as artifact for the expirment

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP Project/saved_models/roberta/',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,
    evaluation_strategy='steps',
    eval_steps=400,
    load_best_model_at_end=True,
    save_total_limit=3,
    save_steps=400

)

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
400,0.199800,0.142141,0.979228,0.935780,0.962264,0.910714
800,0.094600,0.153684,0.976261,0.925926,0.961538,0.892857


<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=845, training_loss=0.14209344852605515, metrics={'train_runtime': 675.6023, 'train_samples_per_second': 9.961, 'train_steps_per_second': 1.251, 'total_flos': 1770737402572800.0, 'train_loss': 0.14209344852605515, 'epoch': 5.0})

### Calculate performance

In [ ]:
eval_result = trainer.evaluate(eval_dataset=val_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 0.1421
eval_accuracy: 0.9792
eval_f1: 0.9358
eval_precision: 0.9623
eval_recall: 0.9107
eval_runtime: 11.6475
eval_samples_per_second: 28.9330
eval_steps_per_second: 3.6920
epoch: 5.0000


In [ ]:
eval_result = trainer.evaluate(eval_dataset=test_dataset)

# Print the evaluation results
print("Evaluation results:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")


<ipython-input-11-14c0bc9f7485>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Evaluation results:
eval_loss: 0.0943
eval_accuracy: 0.9857
eval_f1: 0.9524
eval_precision: 0.9836
eval_recall: 0.9231
eval_runtime: 14.1172
eval_samples_per_second: 29.8220
eval_steps_per_second: 3.7540
epoch: 5.0000
